In [1]:
from FamineStore import FamineStore
import matplotlib.pyplot as plt
import pandas as pd
from api.utils import DataFrameConverters as dfcs

In [2]:
store = FamineStore()

Loading data for all regions
Loading famine data for Awdal


C:\Users\fredd\Anaconda3\envs\scicomp\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\fredd\Anaconda3\envs\scicomp\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Loading famine data for Gedo
Insufficient data for region Gedo, trying to fit with data will give an over-fitted model
Famine dataset loaded for Awdal
Region Gedo not defined, skipping
Using cached StanModel
Reading model fit from stan\fits\FamineModel_Beta_v1_fit_famine_beta_1_Awdal_fa5f488868aaf2077afbcb4bf514c335.fit
Calculating prediction data up to q1/2021
Famine dataset loaded for Awdal
Successfully loaded data, datasets and predictions for regions:
- Awdal
Skipped regions:
- Gedo


In [3]:
store.per_region_data["Awdal"]["food_df"].set_index("Item_Name")

,Date,Region,Market,Item,Price,Year,Month,Quarter
Item_Name,,,,,,,,
Maize (white) - Borama,2015.000000,Awdal,Borama,Maize (white),4000.0,2015,1,1
Maize (white) - Borama,2015.084932,Awdal,Borama,Maize (white),4000.0,2015,2,1
Maize (white) - Borama,2015.161644,Awdal,Borama,Maize (white),4000.0,2015,3,1
Maize (white) - Borama,2015.246575,Awdal,Borama,Maize (white),4000.0,2015,4,2
Maize (white) - Borama,2015.328767,Awdal,Borama,Maize (white),4000.0,2015,5,2
...,...,...,...,...,...,...,...,...
Rice (imported) - Borama,2019.580822,Awdal,Borama,Rice (imported),6250.0,2019,8,3
Rice (imported) - Borama,2019.665753,Awdal,Borama,Rice (imported),6475.0,2019,9,3
Rice (imported) - Borama,2019.747945,Awdal,Borama,Rice (imported),6700.0,2019,10,4


In [4]:
# Monthly
store.per_region_data["Awdal"]["food_df"]

,Date,Region,Market,Item,Price,Year,Month,Quarter
0,2015.000000,Awdal,Borama,Maize (white),4000.0,2015,1,1
1,2015.084932,Awdal,Borama,Maize (white),4000.0,2015,2,1
2,2015.161644,Awdal,Borama,Maize (white),4000.0,2015,3,1
3,2015.246575,Awdal,Borama,Maize (white),4000.0,2015,4,2
4,2015.328767,Awdal,Borama,Maize (white),4000.0,2015,5,2
...,...,...,...,...,...,...,...,...
175,2019.580822,Awdal,Borama,Rice (imported),6250.0,2019,8,3
176,2019.665753,Awdal,Borama,Rice (imported),6475.0,2019,9,3
177,2019.747945,Awdal,Borama,Rice (imported),6700.0,2019,10,4
178,2019.832877,Awdal,Borama,Rice (imported),6500.0,2019,11,4


In [5]:
# Monthly
store.per_region_data["Awdal"]["ffood_df"].shape

(56, 8)

In [6]:
# Daily - Sum over by month
store.per_region_data["Awdal"]["conflict_df"]

,Region,Date,Fatalities,Year,Month,Quarter
0,Awdal,2015.000000,0,2015,1,1
1,Awdal,2015.084932,0,2015,2,1
2,Awdal,2015.161644,0,2015,3,1
3,Awdal,2015.246575,0,2015,4,2
4,Awdal,2015.328767,0,2015,5,2
5,Awdal,2015.413699,0,2015,6,2
6,Awdal,2015.495890,0,2015,7,3
7,Awdal,2015.580822,1,2015,8,3
8,Awdal,2015.665753,0,2015,9,3
9,Awdal,2015.747945,3,2015,10,4


In [7]:
# Daily - take percentage changes?
store.per_region_data["Awdal"]["weather_df"]

,Station,Date,Temperature,Year,Month,Quarter
0,EGAL INTL,2016.248634,76.566667,2016,4,2
1,EGAL INTL,2016.330601,76.566667,2016,5,2
2,EGAL INTL,2016.415301,76.566667,2016,6,2
3,EGAL INTL,2016.497268,76.566667,2016,7,3
4,EGAL INTL,2016.581967,78.400000,2016,8,3
5,EGAL INTL,2016.666667,80.116667,2016,9,3
6,EGAL INTL,2016.748634,77.488889,2016,10,4
7,EGAL INTL,2016.833333,70.482143,2016,11,4
8,EGAL INTL,2016.915301,67.837037,2016,12,4
9,EGAL INTL,2017.000000,68.300000,2017,1,1


In [8]:
# Monthly
store.per_region_pred_data["Awdal"]["food_df"]

,Date,Region,Market,Item,Price,Year,Month,Quarter
0,2015.000000,Awdal,Borama,Maize (white),4000.0,2015,1,1
1,2015.084932,Awdal,Borama,Maize (white),4000.0,2015,2,1
2,2015.161644,Awdal,Borama,Maize (white),4000.0,2015,3,1
3,2015.246575,Awdal,Borama,Maize (white),4000.0,2015,4,2
4,2015.328767,Awdal,Borama,Maize (white),4000.0,2015,5,2
...,...,...,...,...,...,...,...,...
220,2020.833333,Awdal,Borama,Sorghum (red),4000.0,2020,11,4
221,2020.915301,Awdal,Borama,Sorghum (red),4000.0,2020,12,4
222,2021.000000,Awdal,Borama,Sorghum (red),4000.0,2021,1,1
223,2021.084932,Awdal,Borama,Sorghum (red),4000.0,2021,2,1


In [9]:
x = store.per_region_pred_data["Awdal"]["conflict_df"].groupby(["Region", "Year", "Month"], as_index=False).sum()
x.Date = x.Year + (x.Month -1) / 12
x.Quarter = x.Month // 4 + 1
x = x[["Region", "Year", "Month", "Date", "Fatalities", "Quarter"]]
x = x.set_index(["Region", "Year", "Month"])

print(x)
for year in x.index.get_level_values(1).unique():
    for month in range(1, 13):
        try:
            x.loc["Awdal", year, month]
        except KeyError:
            x.loc["Awdal", year, month]=(year + (month - 1) / 12, 0, month // 4 + 1)

x = x.sort_values(by="Date")


C:\Users\fredd\Anaconda3\envs\scicomp\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


KeyError: "['Quarter', 'Fatalities'] not in index"

In [10]:
plt.plot(x.Date, x.Fatalities)

AttributeError: 'DataFrame' object has no attribute 'Fatalities'

In [ ]:
x.query("Year < 2018")

In [11]:

x

,Region,Year,Month,Date
0,Awdal,2015,1,2015.000000
1,Awdal,2015,2,2015.083333
2,Awdal,2015,3,2015.166667
3,Awdal,2015,4,2015.250000
4,Awdal,2015,5,2015.333333
...,...,...,...,...
70,Awdal,2020,11,2020.833333
71,Awdal,2020,12,2020.916667
72,Awdal,2021,1,2021.000000
73,Awdal,2021,2,2021.083333


In [12]:
x = store.per_region_pred_data["Awdal"]["weather_df"].groupby(["Year", "Month"], as_index=False).mean()
x.Date = x.Year + (x.Month - 1) / 12
x.Quarter = x.Month // 4 + 1
x = x[["Year", "Month", "Date", "Temperature", "Quarter"]]
x = x.set_index(["Year", "Month"])
x.index = pd.MultiIndex.from_tuples([(ix[0], int(ix[1])) for ix in x.index.tolist()])
x.index.levels[-1].astype(int)

data = [(2015, 12, 2015 + 11/12, 70, 4)]

i = 0
for year in x.index.get_level_values(0).unique():
    for month in range(1, 13):
        try:
            q = x.loc[year, float(month)]
            data.append((year, month, q[0], q[1], q[2]))
        except KeyError:
            data.append((year, month, year + (month - 1) / 12, data[i][3], month // 4 + 1))
            
    i+=1 
            
df = pd.DataFrame(data, columns = ["Year", "Month", "Date", "Temperature", "Quarter"])        

C:\Users\fredd\Anaconda3\envs\scicomp\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


KeyError: "['Quarter'] not in index"

In [13]:
data

NameError: name 'data' is not defined